In [1]:
%load_ext bigdata
%timeout 300

In [2]:
!hdfs dfs -rm -r -f /tmp/drivers

Deleted /tmp/drivers


In [3]:
!hdfs dfs -rm -r -f /tmp/hive-journals

In [4]:
!hdfs dfs -mkdir /tmp/drivers

In [5]:
#!apt-get update && apt-get install wget

In [6]:
!wget https://raw.githubusercontent.com/jdvelasq/datalabs/master/datasets/drivers/drivers.json

--2021-05-22 02:44:00--  https://raw.githubusercontent.com/jdvelasq/datalabs/master/datasets/drivers/drivers.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4308 (4.2K) [text/plain]
Saving to: 'drivers.json.1'

drivers.json.1      100%[===================>]   4.21K  --.-KB/s    in 0.001s  

2021-05-22 02:44:01 (6.61 MB/s) - 'drivers.json.1' saved [4308/4308]



In [43]:
!hdfs dfs -copyFromLocal drivers.json /tmp/drivers/

copyFromLocal: `/tmp/drivers/drivers.json': File exists


In [8]:
!hdfs dfs -ls /tmp/drivers/

Found 1 items
-rw-r--r--   1 root supergroup       4308 2021-05-22 02:44 /tmp/drivers/drivers.json


In [9]:
%%hive
DROP TABLE IF EXISTS drivers_raw_json;

DROP TABLE IF EXISTS drivers_raw_json;
OK
Time taken: 2.555 seconds


In [11]:
%%hive
CREATE TABLE drivers_raw_json (textcol STRING)
STORED AS TEXTFILE;

CREATE TABLE drivers_raw_json (textcol STRING)
STORED AS TEXTFILE;
OK
Time taken: 0.492 seconds


In [14]:
%%hive
select * from drivers_raw_json;

select * from drivers_raw_json;
OK
Time taken: 0.734 seconds


In [15]:
%%hive
LOAD DATA INPATH '/tmp/drivers/drivers.json'
OVERWRITE INTO TABLE drivers_raw_json;

LOAD DATA INPATH '/tmp/drivers/drivers.json'
OVERWRITE INTO TABLE drivers_raw_json;
Loading data to table default.drivers_raw_json
OK
Time taken: 0.362 seconds


In [17]:
%%hive
select textcol from drivers_raw_json LIMIT 5;

select textcol from drivers_raw_json LIMIT 5;
OK
{"driverId":10,"name":"George Vetticaden","ssn":621011971,"location":"244-4532 Nulla Rd.","certified":"N","wage-plan":"miles"}
{"driverId":11,"name":"Jamie Engesser","ssn":262112338,"location":"366-4125 Ac Street","certified":"N","wage-plan":"miles"}
{"driverId":12,"name":"Paul Coddin","ssn":198041975,"location":"Ap #622-957 Risus. Street","certified":"Y","wage-plan":"hours"}
{"driverId":13,"name":"Joe Niemiec","ssn":139907145,"location":"2071 Hendrerit. Ave","certified":"Y","wage-plan":"hours"}
{"driverId":14,"name":"Adis Cesir","ssn":820812209,"location":"Ap #810-1228 In St.","certified":"Y","wage-plan":"hours"}
Time taken: 0.071 seconds, Fetched: 5 row(s)


In [ ]:
#GET_JSON_OBJECT --> sIRVE PARA HACER FILTROS EN REGISTROS UNICOS (sIN COLUMNAS) QUE FUERON TRAIDOS DE ARCHIVOS JSON
#JSON_TUPLE --> sIRVE PARA HACER FILTROS EN REGISTROS UNICOS (sIN COLUMNAS) QUE FUERON TRAIDOS DE ARCHIVOS JSON

In [18]:
%%hive
select GET_JSON_OBJECT(textcol, '$.driverId'),
       GET_JSON_OBJECT(textcol, '$.name'),
       GET_JSON_OBJECT(textcol, '$.ssn'),
       GET_JSON_OBJECT(textcol, '$.certified')
    
    from drivers_raw_json limit 5;
    

select GET_JSON_OBJECT(textcol, '$.driverId'),
       GET_JSON_OBJECT(textcol, '$.name'),
       GET_JSON_OBJECT(textcol, '$.ssn'),
       GET_JSON_OBJECT(textcol, '$.certified')
    from drivers_raw_json limit 5;
OK
10	George Vetticaden	621011971	N
11	Jamie Engesser	262112338	N
12	Paul Coddin	198041975	Y
13	Joe Niemiec	139907145	Y
14	Adis Cesir	820812209	Y
Time taken: 0.216 seconds, Fetched: 5 row(s)


In [23]:
%%hive
select jt.driverId,
      jt.name,
      jt.ssn,
      jt.certified
from drivers_raw_json tx
    LATERAL VIEW 
                JSON_TUPLE(tx.textcol,'driverId','name','ssn','certified') jt
                as driverId, name, ssn, certified
            limit 5;

select jt.driverId,
      jt.name,
      jt.ssn,
      jt.certified
from drivers_raw_json tx
    LATERAL VIEW 
 jt             JSON_TUPLE(tx.textcol,'driverId','name','ssn','certified') 
                as driverId, name, ssn, certified
            limit 5;
OK
10	George Vetticaden	621011971	N
11	Jamie Engesser	262112338	N
12	Paul Coddin	198041975	Y
13	Joe Niemiec	139907145	Y
14	Adis Cesir	820812209	Y
Time taken: 0.045 seconds, Fetched: 5 row(s)


In [29]:
%%hive
DROP TABLE IF EXISTS TABLAJSON;

DROP TABLE IF EXISTS TABLAJSON;
OK
Time taken: 0.19 seconds


In [30]:
%%hive
CREATE TABLE TABLAJSON AS
select jt.driverId,
      jt.name,
      jt.ssn,
      jt.certified
from drivers_raw_json tx
    LATERAL VIEW 
                JSON_TUPLE(tx.textcol,'driverId','name','ssn','certified') jt
                as driverId, name, ssn, certified
            ;

CREATE TABLE TABLAJSON AS
select jt.driverId,
      jt.name,
      jt.ssn,
      jt.certified
from drivers_raw_json tx
    LATERAL VIEW 
 jt             JSON_TUPLE(tx.textcol,'driverId','name','ssn','certified') 
                as driverId, name, ssn, certified
            ;
Query ID = root_20210522043000_9ebcc191-f6dd-483c-9fb7-a1f58658e6c5
Total jobs = 3
Launching Job 1 out of 3
Number of reduce tasks is set to 0 since there's no reduce operator
Starting Job = job_1621642324979_0004, Tracking URL = http://d105d69c1e79:8088/proxy/application_1621642324979_0004/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1621642324979_0004
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 0
2021-05-22 04:30:05,618 Stage-1 map = 0%,  reduce = 0%
2021-05-22 04:30:09,722 Stage-1 map = 100%,  reduce = 0%, Cumulative CPU 0.89 sec
MapReduce Total cumulative CPU time: 890 msec
Ended Job = job_1621642324979_0004
Stage-4 is selected by condition resolver.
Stage-3 is 

In [31]:
%%hive
select * from TABLAJSON;

select * from TABLAJSON;
OK
10	George Vetticaden	621011971	N
11	Jamie Engesser	262112338	N
12	Paul Coddin	198041975	Y
13	Joe Niemiec	139907145	Y
14	Adis Cesir	820812209	Y
15	Rohit Bakshi	239005227	Y
16	Tom McCuch	363303105	Y
17	Eric Mizell	123808238	Y
18	Grant Liu	171010151	Y
19	Ajay Singh	160005158	Y
20	Chris Harris	921812303	Y
21	Jeff Markham	209408086	Y
22	Nadeem Asghar	783204269	Y
23	Adam Diaz	928312208	Y
24	Don Hilborn	254412152	Y
25	Jean-Philippe Playe	913310051	Y
26	Michael Aube	124705141	Y
27	Mark Lochbihler	392603159	Y
28	Olivier Renault	959908181	Y
29	Teddy Choi	185502192	Y
30	Dan Rice	282307061	Y
31	Rommel Garcia	858912101	Y
32	Ryan Templeton	290304287	Y
33	Sridhara Sabbella	967409015	Y
34	Frank Romano	391407216	Y
35	Emil Siemes	971401151	Y
36	Andrew Grande	245303216	Y
37	Wes Floyd	190504074	Y
38	Scott Shaw	386411175	Y
39	David Kaiser	967706052	Y
40	Nicolas Maillard	208510217	Y
41	Greg Phillips	308103116	Y
42	Randy Gelhausen	853302254	Y
43	Dave Patton	977706052	Y
Time taken:

In [32]:
%%hive
select * from TABLAJSON where certified='Y';

select * from TABLAJSON where certified='Y';
OK
12	Paul Coddin	198041975	Y
13	Joe Niemiec	139907145	Y
14	Adis Cesir	820812209	Y
15	Rohit Bakshi	239005227	Y
16	Tom McCuch	363303105	Y
17	Eric Mizell	123808238	Y
18	Grant Liu	171010151	Y
19	Ajay Singh	160005158	Y
20	Chris Harris	921812303	Y
21	Jeff Markham	209408086	Y
22	Nadeem Asghar	783204269	Y
23	Adam Diaz	928312208	Y
24	Don Hilborn	254412152	Y
25	Jean-Philippe Playe	913310051	Y
26	Michael Aube	124705141	Y
27	Mark Lochbihler	392603159	Y
28	Olivier Renault	959908181	Y
29	Teddy Choi	185502192	Y
30	Dan Rice	282307061	Y
31	Rommel Garcia	858912101	Y
32	Ryan Templeton	290304287	Y
33	Sridhara Sabbella	967409015	Y
34	Frank Romano	391407216	Y
35	Emil Siemes	971401151	Y
36	Andrew Grande	245303216	Y
37	Wes Floyd	190504074	Y
38	Scott Shaw	386411175	Y
39	David Kaiser	967706052	Y
40	Nicolas Maillard	208510217	Y
41	Greg Phillips	308103116	Y
42	Randy Gelhausen	853302254	Y
43	Dave Patton	977706052	Y
Time taken: 0.08 seconds, Fetched: 32 row(s)


In [ ]:
##Serde --> Funciona como un driver de conexion

In [36]:
%%hive
select textcol from drivers_raw_json LIMIT 5;

select textcol from drivers_raw_json LIMIT 5;
OK
{"driverId":10,"name":"George Vetticaden","ssn":621011971,"location":"244-4532 Nulla Rd.","certified":"N","wage-plan":"miles"}
{"driverId":11,"name":"Jamie Engesser","ssn":262112338,"location":"366-4125 Ac Street","certified":"N","wage-plan":"miles"}
{"driverId":12,"name":"Paul Coddin","ssn":198041975,"location":"Ap #622-957 Risus. Street","certified":"Y","wage-plan":"hours"}
{"driverId":13,"name":"Joe Niemiec","ssn":139907145,"location":"2071 Hendrerit. Ave","certified":"Y","wage-plan":"hours"}
{"driverId":14,"name":"Adis Cesir","ssn":820812209,"location":"Ap #810-1228 In St.","certified":"Y","wage-plan":"hours"}
Time taken: 0.052 seconds, Fetched: 5 row(s)


In [35]:
%%hive
DROP TABLE IF EXISTS drivers_json_serde;

DROP TABLE IF EXISTS drivers_json_serde;
OK
Time taken: 0.004 seconds


In [38]:
%%hive
CREATE TABLE drivers_json_serde (
    driverId INT,
    name STRING,
    ssn BIGINT, 
    location STRING, 
    certified STRING,
    wageplan STRING)
ROW FORMAT SERDE 'org.apache.hive.hcatalog.data.JsonSerDe'
STORED AS TEXTFILE
LOCATION '/tmp/drivers-json';

CREATE TABLE drivers_json_serde (
    driverId INT,
    name STRING,
    ssn BIGINT, 
    location STRING, 
    certified STRING,
    wageplan STRING)
ROW FORMAT SERDE 'org.apache.hive.hcatalog.data.JsonSerDe'
STORED AS TEXTFILE
LOCATION '/tmp/drivers-json';
OK
Time taken: 0.037 seconds


In [39]:
%%hive
select * from drivers_json_serde;

select * from drivers_json_serde;
OK
Time taken: 0.051 seconds


In [44]:
%%hive
LOAD DATA INPATH '/tmp/drivers/drivers.json' OVERWRITE INTO TABLE drivers_json_serde;

json_serde;NPATH '/tmp/drivers/drivers.json' OVERWRITE INTO TABLE drivers_ 
Loading data to table default.drivers_json_serde
OK
Time taken: 0.169 seconds


In [46]:
%%hive
select * from drivers_json_serde;

select * from drivers_json_serde;
OK
10	George Vetticaden	621011971	244-4532 Nulla Rd.	N	NULL
11	Jamie Engesser	262112338	366-4125 Ac Street	N	NULL
12	Paul Coddin	198041975	Ap #622-957 Risus. Street	Y	NULL
13	Joe Niemiec	139907145	2071 Hendrerit. Ave	Y	NULL
14	Adis Cesir	820812209	Ap #810-1228 In St.	Y	NULL
15	Rohit Bakshi	239005227	648-5681 Dui- Rd.	Y	NULL
16	Tom McCuch	363303105	P.O. Box 313- 962 Parturient Rd.	Y	NULL
17	Eric Mizell	123808238	P.O. Box 579- 2191 Gravida. Street	Y	NULL
18	Grant Liu	171010151	Ap #928-3159 Vestibulum Av.	Y	NULL
19	Ajay Singh	160005158	592-9430 Nonummy Avenue	Y	NULL
20	Chris Harris	921812303	883-2691 Proin Avenue	Y	NULL
21	Jeff Markham	209408086	Ap #852-7966 Facilisis St.	Y	NULL
22	Nadeem Asghar	783204269	154-9147 Aliquam Ave	Y	NULL
23	Adam Diaz	928312208	P.O. Box 260- 6127 Vitae Road	Y	NULL
24	Don Hilborn	254412152	4361 Ac Road	Y	NULL
25	Jean-Philippe Playe	913310051	P.O. Box 812- 6238 Ac Rd.	Y	NULL
26	Michael Aube	124705141	P.O. Box 213- 8948 Nec Ave	Y	